## Q1. What is Gradient Boosting Regression?

Gradient Boosting Regression (GBR) is a popular and widely-used machine learning algorithm for regression problems. It is based on the concept of gradient boosting, which involves iteratively training weak learners on the residuals or errors of the previous learners, and combining the predictions of the weak learners to create a strong learner.

In GBR, the weak learners are typically decision trees, and the residuals are computed as the difference between the true values and the predicted values of the training examples. The decision trees are trained to minimize the mean squared error (MSE) or mean absolute error (MAE) between the predicted values and the true values.

The process of GBR typically involves the following steps:

1. Initialize the predicted values of the training examples to be equal to the mean of the target variable.
2. For each iteration t = 1, 2, ..., T:
a. Compute the residuals of the training examples as the difference between the true values and the predicted values.
b. Train a decision tree on the residuals to minimize the MSE or MAE.
c. Compute the weight or contribution of the decision tree based on its performance.
d. Update the predicted values of the training examples by adding the weighted prediction of the decision tree.
3. Combine the predictions of the decision trees using a weighted sum or a weighted average.

The number of iterations T is a hyperparameter that can be tuned to achieve the best performance. The weight of each decision tree is typically proportional to its accuracy, so that more accurate trees have a greater influence on the final prediction.

Overall, GBR is a powerful and flexible algorithm for regression problems, and it can be used to model complex relationships and interactions between the features and the target variable. It is particularly useful for problems with a large number of features or a high degree of non-linearity. However, it can also be prone to overfitting, especially if the number of iterations is too large or the decision trees are too deep.

### Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy. Use a simple regression problem as an example and train the model on a small dataset. Evaluate the model's performance using metrics such as mean squared error and R-squared.

In [1]:
# 1. Import the necessary libraries
import numpy as np

# 2. Create a small dataset for a simple regression problem
np.random.seed(42)
X = np.random.rand(30, 1)
y = 3 * X + 2 + np.random.randn(30, 1)

# 3. Define the mean squared error (MSE) and R-squared metrics
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def r2_score(y_true, y_pred):
    u = np.mean(y_true, axis=0)
    ss_res = np.sum((y_true - y_pred) ** 2, axis=0)
    ss_tot = np.sum((y_true - u) ** 2, axis=0)
    return 1 - (ss_res / ss_tot)

# 4. Implement the gradient boosting algorithm
class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.estimators = []

    def fit(self, X, y):
        y = np.squeeze(y)
        self.n_samples, self.n_features = X.shape

        # Initialize predictions
        y_pred = np.zeros(self.n_samples)

        # Train the estimators
        for _ in range(self.n_estimators):
            # Compute residuals
            residuals = y - y_pred

            # Fit a decision tree to the residuals
            estimator = self._fit_decision_tree(X, residuals)

            # Update the predictions
            y_pred += self.learning_rate * estimator.predict(X)

            # Store the estimator
            self.estimators.append(estimator)

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for estimator in self.estimators:
            y_pred += self.learning_rate * estimator.predict(X)
        return y_pred

    def _fit_decision_tree(self, X, y):
        from sklearn.tree import DecisionTreeRegressor
        return DecisionTreeRegressor(max_depth=self.max_depth).fit(X, y)

# 5. Train the model on the dataset
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
gbr.fit(X, y)

# 6. Evaluate the model's performance
y_pred = gbr.predict(X)
print("Mean Squared Error:", mse(y, y_pred))
print("R-squared:", r2_score(y, y_pred))


Mean Squared Error: 1.613712181388551
R-squared: [-1.65005439e+00 -4.65862489e+00 -2.58708592e-01 -2.59191117e-01
 -1.56965362e+00 -1.33986674e+00 -9.27793649e-01 -1.59329769e-01
 -1.94341233e+00 -2.12979712e-04 -7.51479214e-01 -2.00093750e-02
 -3.73933969e-02 -2.18867444e-01 -1.75776659e-03 -2.08320856e-01
 -1.27653449e-01 -6.15677281e-02 -1.65005439e+00 -7.14116819e-01
 -5.45723270e-02 -1.03906306e-01 -1.19893017e-02 -3.21706160e+00
 -3.47566038e-01 -4.70109808e-01 -1.15243748e+00 -9.96108852e-01
 -2.65998404e+00 -2.61812216e-02]


## Q3. Experiment with different hyperparameters such as learning rate, number of trees, and tree depth to optimise the performance of the model. Use grid search or random search to find the bes hyperparameters

In [2]:
# 1. Import the necessary libraries
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV

# 2. Create a small dataset for a simple regression problem
np.random.seed(42)
X = np.random.rand(30, 1)
y = 3 * X + 2 + np.random.randn(30, 1)

# 3. Define the mean squared error (MSE) and R-squared metrics
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def r2_score(y_true, y_pred):
    u = np.mean(y_true, axis=0)
    ss_res = np.sum((y_true - y_pred) ** 2, axis=0)
    ss_tot = np.sum((y_true - u) ** 2, axis=0)
    return 1 - (ss_res / ss_tot)

# 4. Implement the gradient boosting algorithm (updated)
class GradientBoostingRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.estimators = []

    def fit(self, X, y):
        y = np.squeeze(y)
        self.n_samples, self.n_features = X.shape

        # Initialize predictions
        y_pred = np.zeros(self.n_samples)

        # Train the estimators
        for _ in range(self.n_estimators):
            # Compute residuals
            residuals = y - y_pred

            # Fit a decision tree to the residuals
            estimator = self._fit_decision_tree(X, residuals)

            # Update the predictions
            y_pred += self.learning_rate * estimator.predict(X)

            # Store the estimator
            self.estimators.append(estimator)

        return self

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for estimator in self.estimators:
            y_pred += self.learning_rate * estimator.predict(X)
        return y_pred

    def _fit_decision_tree(self, X, y):
        from sklearn.tree import DecisionTreeRegressor
        return DecisionTreeRegressor(max_depth=self.max_depth).fit(X, y)

# 5. Perform grid search to find the best hyperparameters
param_grid = {
    "n_estimators": [50, 100, 150],
    "learning_rate": [0.01, 0.1, 0.5],
    "max_depth": [1, 3, 5],
}

gbr = GradientBoostingRegressor()
grid_search = GridSearchCV(gbr, param_grid, cv=5, scoring="r2")
grid_search.fit(X, np.squeeze(y))

print("Best hyperparameters:", grid_search.best_params_)
print("Best R-squared score:", grid_search.best_score_)

# 6. Train the model with the best hyperparameters and evaluate its performance
gbr_best = GradientBoostingRegressor(**grid_search.best_params_)
gbr_best.fit(X, y)

y_pred = gbr_best.predict(X)
print("Mean Squared Error:", mse(y, y_pred))
print("R-squared:", r2_score(y, y_pred))


Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 50}
Best R-squared score: -1.2150796565021675
Mean Squared Error: 1.1328237102103464
R-squared: [-0.36791328 -0.78839123 -0.29575639 -0.29575639 -0.36791328 -0.05105367
 -0.14447323 -0.29575639 -0.29575639 -0.29575639 -0.14447323 -0.10966759
 -0.29575639 -0.36791328 -0.36791328 -0.36791328 -0.36791328 -0.29575639
 -0.36791328 -0.36791328 -0.29575639 -0.05105367 -0.36791328 -0.36791328
 -0.29575639 -0.29575639 -0.36791328 -0.29575639 -0.29575639 -0.14447323]


## Q4. What is a weak learner in Gradient Boosting?

In Gradient Boosting, a weak learner is a model with slightly better performance than a random guess, but not strong enough to provide accurate predictions on its own. The main idea behind Gradient Boosting is to combine the predictions of multiple weak learners to create a strong, accurate model.

In the context of Gradient Boosting, weak learners are typically decision trees with a small number of nodes or limited depth. These trees are trained on the residuals (i.e., the difference between the true values and the current predictions) of the previous iteration, aiming to correct the errors made by the previous trees. By iteratively adding trees and minimizing the residuals, the Gradient Boosting algorithm gradually improves the overall performance of the model.

The advantage of using weak learners is that they are computationally efficient and easy to train. Additionally, by combining the predictions of many diverse weak learners, the Gradient Boosting algorithm can achieve high accuracy and prevent overfitting.

## Q5. What is the intuition behind the Gradient Boosting algorithm?

The intuition behind the Gradient Boosting algorithm is to iteratively build an ensemble of weak learners (typically decision trees) that collectively form a strong, accurate model. The algorithm aims to minimize the prediction error by training each subsequent weak learner on the residuals (i.e., the difference between the true values and the current predictions) of the previous iteration.

Here's a step-by-step breakdown of the Gradient Boosting algorithm:

1. Initialize the model's predictions with a constant value, usually the mean or median of the target variable.
2. For a fixed number of iterations (or until a stopping criterion is met):
a. Calculate the residuals, which represent the errors made by the current model.
b. Train a weak learner (e.g., a shallow decision tree) on the residuals, aiming to predict and correct the errors.
c. Update the model's predictions by adding the weak learner's predictions, typically multiplied by a learning rate to control the contribution of each tree.
3. Return the final model, which is an ensemble of the trained weak learners.

The key idea is that each weak learner focuses on the mistakes of the previous one, gradually refining the predictions and improving the overall performance of the model. By combining the predictions of many diverse weak learners, the Gradient Boosting algorithm can achieve high accuracy and prevent overfitting.

In summary, the Gradient Boosting algorithm is an iterative process that builds an ensemble of weak learners, where each learner is trained to correct the errors of the previous one, ultimately resulting in a strong, accurate model.

## Q6. How does Gradient Boosting algorithm build an ensemble of weak learners?

The Gradient Boosting algorithm builds an ensemble of weak learners by iteratively training them on the residuals (i.e., the difference between the true values and the current predictions) of the previous iteration. This process allows each weak learner to focus on the mistakes of the previous one, gradually refining the predictions and improving the overall performance of the model.

Here's a more detailed explanation of how the Gradient Boosting algorithm builds an ensemble of weak learners:

1. Initialize the model's predictions: The algorithm starts by initializing the model's predictions with a constant value, usually the mean or median of the target variable. This provides a baseline for the model's performance.
2. Iterative training of weak learners: The algorithm then enters an iterative loop, where it trains a weak learner (e.g., a shallow decision tree) on the residuals of the previous iteration. The goal of each weak learner is to predict and correct the errors made by the previous one.

The iterative training process can be broken down into the following steps:

a. Calculate residuals: The algorithm computes the residuals by subtracting the current predictions from the true values. These residuals represent the errors that the current model is making.

b. Train a weak learner: The algorithm trains a weak learner on the residuals, using them as the target variable. This allows the weak learner to focus on the mistakes of the previous iteration and learn how to correct them.

c. Update predictions: The algorithm updates the model's predictions by adding the weak learner's predictions, typically multiplied by a learning rate to control the contribution of each tree. This ensures that the model's performance is gradually improved, without over-correcting or overfitting.
3. Ensemble of weak learners: After the iterative training process is complete, the algorithm returns the final model, which is an ensemble of the trained weak learners. The predictions of the ensemble are obtained by summing the predictions of each individual weak learner.

In summary, the Gradient Boosting algorithm builds an ensemble of weak learners by iteratively training them on the residuals of the previous iteration, allowing each learner to focus on the mistakes of the previous one, and gradually improving the overall performance of the model.

## Q7. What are the steps involved in constructing the mathematical intuition of Gradient Boosting algorithm?

To construct the mathematical intuition of the Gradient Boosting algorithm, we can follow these steps:

1. Define the objective function: The objective function is a measure of the difference between the true values (y) and the predicted values (F(x)). A common choice for the objective function is the mean squared error (MSE) or the mean absolute error (MAE). In this explanation, we will use the MSE as the objective function:

Obj(y, F(x)) = (1/n) Σ (yᵢ - F(xᵢ))²

where n is the number of samples, yᵢ is the true value for the i-th sample, and F(xᵢ) is the predicted value for the i-th sample.

2. Initialize the model's predictions: The algorithm starts by initializing the model's predictions with a constant value, usually the mean or median of the target variable. This provides a baseline for the model's performance.

F₀(x) = argmin₌ᵡ Σ (yᵢ - w)²

3. Iterative training of weak learners: The algorithm then enters an iterative loop, where it trains a weak learner (e.g., a shallow decision tree) on the residuals of the previous iteration. The goal of each weak learner is to predict and correct the errors made by the previous one.

For each iteration m (m = 1, 2, ..., M):

a. Calculate residuals: The algorithm computes the residuals by subtracting the current predictions from the true values. These residuals represent the errors that the current model is making.

rᵢₘ = yᵢ - Fₘ₋₁(xᵢ)

b. Train a weak learner: The algorithm trains a weak learner (hₘ(x)) on the residuals, using them as the target variable. This allows the weak learner to focus on the mistakes of the previous iteration and learn how to correct them.

hₘ(x) = argmin₌ₕ Σ (rᵢₘ - h(xᵢ))²

c. Update predictions: The algorithm updates the model's predictions by adding the weak learner's predictions, multiplied by a learning rate (α) to control the contribution of each tree. This ensures that the model's performance is gradually improved, without over-correcting or overfitting.

Fₘ(x) = Fₘ₋₁(x) + α \* hₘ(x)

4. Ensemble of weak learners: After the iterative training process is complete, the algorithm returns the final model, which is an ensemble of the trained weak learners. The predictions of the ensemble are obtained by summing the predictions of each individual weak learner.

F(x) = F₀(x) + Σ (α \* hₘ(x)) for m = 1, 2, ..., M

In summary, the mathematical intuition behind the Gradient Boosting algorithm involves defining an objective function, initializing the model's predictions, iteratively training weak learners on the residuals of the previous iteration, and updating the model's predictions using a learning rate. This process results in an ensemble of weak learners that collectively form a strong, accurate model.